In [1]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

In [4]:
df = pd.read_csv('songs_all.csv',encoding='latin1')
df = df.drop(['Unnamed: 0'], axis = 1)
df.head()

,album_artist,album_mood,album_title,allmusic_href,lyrics,lyrics(normalized_no_stopwords),lyrics(normalized_with_stopwords),lyrics_href,track_genres,track_length,track_moods,track_title,track_url
0,Primal Scream,ambitious,Screamadelica,/album/screamadelica-mw0000270548,"I was blind, now I can see\r\r\r\r\nYou made a...",blind see make believer blind see make believe...,i be blind now i can see you make a believer o...,/lyric/7390644/Primal+Scream/Movin%27+on+Up+%5...,"['Electronic', 'Pop/Rock']",3:51,"['carefree', 'earnest', 'exuberant', 'stylish'...",Movin' on Up,https://www.allmusic.com/song/movin-on-up-mt00...
1,Primal Scream,ambitious,Screamadelica,/album/screamadelica-mw0000270548,Gonna dance to the music all night long\r\r\r\...,go dance music night long gettin high gettin h...,go to dance to the music all night long gettin...,/lyric/999573/Primal+Scream/Don%27t+Fight+It%2...,"['Electronic', 'Pop/Rock']",6:53,"['carefree', 'earnest', 'exuberant', 'stylish'...","Don't Fight It, Feel It",https://www.allmusic.com/song/dont-fight-it-fe...
2,Primal Scream,ambitious,Screamadelica,/album/screamadelica-mw0000270548,My brightest star's my inner light\r\r\r\r\nLe...,bright star inner light let guide experience i...,my bright star my inner light let it guide me ...,/lyric/999574/Primal+Scream/Higher+Than+the+Sun,"['Electronic', 'Pop/Rock']",3:38,"['ambitious', 'lush', 'trippy', 'carefree', 'e...",Higher Than the Sun,https://www.allmusic.com/song/higher-than-the-...
3,Primal Scream,ambitious,Screamadelica,/album/screamadelica-mw0000270548,Come together oh oh\r\r\r\r\nCome together oh ...,come together oh oh come together oh oh come t...,come together oh oh come together oh oh come t...,/lyric/4633273/Primal+Scream/Come+Together+%5B...,"['Electronic', 'Pop/Rock']",10:21,"['ambitious', 'carefree', 'earnest', 'exuberan...",Come Together,https://www.allmusic.com/song/come-together-mt...
4,Primal Scream,ambitious,Screamadelica,/album/screamadelica-mw0000270548,Just what is it that you want to do?\r\r\r\r\n...,want well wanna free wanna free wanna wanna ge...,just what be it that you want to do well we wa...,/lyric/999577/Primal+Scream/Loaded,"['Electronic', 'Pop/Rock']",7:02,"['celebratory', 'trippy', 'carefree', 'earnest...",Loaded,https://www.allmusic.com/song/loaded-mt0012027546


In [13]:
l_neg = []
l_pos = []
l_neu = []
l_compound = []
l_label = []

analyzer = SentimentIntensityAnalyzer()
norm_lyrics = df['lyrics(normalized_no_stopwords)']

for lyrics in norm_lyrics:
    vs = analyzer.polarity_scores(lyrics)
    l_neg.append(vs['neg'])
    l_pos.append(vs['pos'])
    l_neu.append(vs['pos'])
    l_compound.append(vs['compound'])
    if (vs['compound'] > 0):
        l_label.append(1)
    else:
        l_label.append(0)
        
titles = df['track_title'].tolist()
score_df = pd.DataFrame([titles, l_neg, l_pos, l_neu, l_compound, l_label],
                        index = (['title', 'neg_score', 'pos_score', 'neu_score', 'compound_score', 'sentiment_label'])).T

score_df.head()

,title,neg_score,pos_score,neu_score,compound_score,sentiment_label
0,Movin' on Up,0.345,0,0,-0.9864,0
1,"Don't Fight It, Feel It",0.192,0.09,0.09,-0.836,0
2,Higher Than the Sun,0.029,0.233,0.233,0.9552,1
3,Come Together,0,0.344,0.344,0.9894,1
4,Loaded,0.263,0.361,0.361,0.8458,1


In [14]:
df.insert(loc=df.columns.get_loc("lyrics(normalized_with_stopwords)")+1, column='positive_score', value=l_pos)
df.insert(loc=df.columns.get_loc("lyrics(normalized_with_stopwords)")+2, column='negative_score', value=l_neg)
df.insert(loc=df.columns.get_loc("lyrics(normalized_with_stopwords)")+3, column='neutral_score', value=l_neu)
df.insert(loc=df.columns.get_loc("lyrics(normalized_with_stopwords)")+4, column='compound_score', value=l_compound)
df.insert(loc=df.columns.get_loc("lyrics(normalized_with_stopwords)")+5, column='sentiment_label', value=l_label)
df.to_csv('songs_all_labeled.csv')

In [24]:
import pandas as pd
songs = pd.read_csv('songs_all_labeled.csv')
songs = songs[['lyrics(normalized_with_stopwords)','sentiment_label']]
lyrics_positive = songs['lyrics(normalized_with_stopwords)'][songs['sentiment_label']==1]
corpus_positive = ' '.join(lyrics_positive)
lyrics_negative = songs['lyrics(normalized_with_stopwords)'][songs['sentiment_label']==0]
corpus_negative = ' '.join(lyrics_negative)
with open('corpus_positive.txt', 'w') as f:
    f.write(corpus_positive)
with open('corpus_negative.txt', 'w') as f:
    f.write(corpus_negative)



In [20]:
lyrics_positive.shape

(704,)

In [21]:
lyrics_negative.shape

(396,)

In [22]:
len(corpus_negative)

353158

In [23]:
len(corpus_positive)

666214